In [ ]:
import sys
import os
sys.path.append(os.path.abspath('') + "/../src")
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from pyspark.sql import functions as F
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
from scipy.optimize import minimize

def predict_viscosity_U(Q,T,parm_B,parm_y):
    B = parm_B
    y = parm_y
    U = Q/B*(T/25)**y
    return U

parm_B_y_init = (1, 1.5)
Q = df['Q'].values
T = df['T'].values
U_actual = df['U'].values

def cost_function(_x0):
    B = _x0[0]
    y = _x0[1]
    U_predicted = predict_viscosity_U(Q, T, B, y)
    return np.mean((U_predicted - U_actual)**2)**0.5

# Define the bounds for B and y
bounds = [(0, 5), (0, 5)]

# Define the initial guess for B and y
x0 = parm_B_y_init

# Use minimize function to find the best possible combinations for B and y
result = minimize(cost_function, x0, bounds=bounds, tol=1e-10, method='SLSQP')

result.x

In [ ]:
# Initialize parameters
B_init = 2
y_init = 1
B_constraint = [0,5]
y_constraint = [0,5]
learning_rate = 0.000001
max_iter = 10000

# Convert pandas dataframe to numpy arrays
Q = df['Q'].values
T = df['T'].values
U_actual = df['U'].values

# Define cost function
def cost_function(B, y):
    U_predicted = predict_viscosity_U(Q, T, B, y)
    return np.mean((U_predicted - U_actual)**2)

# Gradient descent algorithm
B = B_init
y = y_init
cost_history = []
B_history = []
y_history = []

for i in range(max_iter):
    U_predicted = predict_viscosity_U(Q, T, B, y)
    error = U_predicted - U_actual
    dJdB = - 2 * np.mean(error * (Q / B) * (T / 25)**y) * B**(-2)
    dJdy = 2 * np.mean(error * (Q / B) * (T / 25)**y * np.log(T / 25))
    
    if (B - learning_rate * dJdB) > B_constraint[0] and (B - learning_rate * dJdB) < B_constraint[1]:
        B = B - learning_rate * dJdB
    if (y - learning_rate * dJdy) > y_constraint[0] and (B - learning_rate * dJdy) < y_constraint[1]:
        y = y - learning_rate * dJdy
    
    cost = cost_function(B, y)
    cost_history.append(cost)
    B_history.append(B)
    y_history.append(y)

# Plot cost history
import matplotlib.pyplot as plt
plt.plot(cost_history,'k')
plt.xlabel('Iteration')
plt.ylabel('RMSE (mPas)')
plt.show()

plt.plot(B_history,cost_history,'k')
plt.xlabel('B')
plt.ylabel('RMSE (mPas)')
plt.show()

plt.plot(y_history,cost_history,'k')
plt.xlabel('y')
plt.ylabel('RMSE (mPas)')
plt.show()

B,y,cost,i

In [ ]:
# Define the range and step size for B and y
B_range = np.arange(1, 2, 0.001)
y_range = np.arange(1, 2.5, 0.001)

# Initialize the cost matrix
cost_matrix = np.zeros((len(B_range), len(y_range)))

# Perform grid search
for i, B in enumerate(B_range):
    for j, y in enumerate(y_range):
        U_predicted = predict_viscosity_U(Q, T, B, y)
        error = U_predicted - U_actual
        cost = np.mean((error)**2)
        cost_matrix[i, j] = cost

min_indices = np.unravel_index(np.argmin(cost_matrix), cost_matrix.shape)
global_min_B = B_range[min_indices[0]]
global_min_y = y_range[min_indices[1]]

global_min_B, global_min_y, cost_matrix.min()

In [ ]:
# Create the 3D surface plot
fig = go.Figure(data=[go.Surface(x=B_range, y=y_range, z=cost_matrix.T, colorscale='viridis', cmin=0)])
fig.update_layout(title='Cost RMSE vs B and y', scene=dict(xaxis_title='B', yaxis_title='y', zaxis_title='Cost RMSE'), width=800, height=800)

In [ ]:
plt.figure(figsize=(10,8))
contour = plt.contourf(B_range, y_range, cost_matrix.T, levels=25, cmap='viridis')
plt.xlabel('B')
plt.ylabel('y')
plt.title('Finding optimal parameters for B and y to get lowest cost-function')
plt.colorbar(contour)
plt.plot(B_history,y_history,'w-',label='Gradient Descent path')
plt.legend()
plt.show()